## Imports 

In [1]:
from SETUP import *
from REDUCE_FEATURES import *
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
data_file = '/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001/20240-0001-Data.tsv'
filenames = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_group_file_names.txt'
csv_root_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_name_data/'

In [3]:
suicidality_features = ['V01995', 'V01997', 'V01999', 'V02044', 'V02003', 'V02004', 'V02009', 'V02023', 'V02025', 'V02027', 'V02029', 'V02031', 'V02032', 'V02035', 'V02036', 'V02041']

In [4]:
set_inst = Setup(csv_root_path, filenames)
full_dict = set_inst.execute_setup()
reduce_inst = Reduce_Features(data_file, full_dict)
dirty_df = reduce_inst.execute_reduce()

In [5]:
df = dirty_df.drop(dirty_df[dirty_df['V01993'] == ' '].index)

In [6]:
## Turn strings to ints 
mask = {str(num): num for num in range(200000)}

In [7]:
mask[' '] = 0

In [8]:
mask['-9'] = -9

In [9]:
mask['-8'] = -8

In [10]:
mask[None] = 0

In [11]:
for idx, feature in enumerate(list(df)): 
    df[feature] = df[feature].map(mask)
    if idx % 250 == 0: 
        print('completed feature {} of 2110'.format(idx))

completed feature 0 of 2110
completed feature 250 of 2110
completed feature 500 of 2110
completed feature 750 of 2110
completed feature 1000 of 2110


In [12]:
df.drop(suicidality_features, axis=1, inplace=True)

In [13]:
df.head()

,V01638,V01639,V01643,V01644,V01646,V01647,V01648,V01649,V01650,V01651,...,V08495,V08549,V07725,V07894,V08501,V08500,V08553,V07750,V07748,V07899
0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,0,0,0,0,0,0,0,0,0,0,...,5,5,1,1,5,5,5,5,5,5
3,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
5,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
6,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5


In [14]:
df = df.dropna(axis=1)

In [15]:
df['V01993'].describe()

count    15890.000000
mean         4.456765
std          1.429194
min         -9.000000
25%          5.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: V01993, dtype: float64

In [16]:
X_df = df.loc[:, df.columns != 'V01993']
y_df = df['V01993'].map({5:0, 1:1, -9:0, -8:0})

In [17]:
X = X_df.as_matrix()
y = y_df.as_matrix()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(11123, 1192) (11123,)
(4767, 1192) (4767,)


In [20]:
def print_scores(y_test, predictions):
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('Precision:', precision_score(y_test, predictions))
    print('Recall:', recall_score(y_test, predictions))
    return None

# Step 3  - MODELING


In [81]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score

## Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve

In [ ]:
# DEFAULT BASE CLASSIFIER SETTINGS 
# clf = RandomForestClassifier(bootstrap= True,
#  class_weight= None,
#  criterion= 'gini',
#  max_depth= 10,
#  max_features= 'auto',
#  max_leaf_nodes= None,
#  min_impurity_split= 1e-07,
#  min_samples_leaf= 1,
#  min_samples_split= 2,
#  min_weight_fraction_leaf= 0.0,
#  n_estimators= 10,
#  n_jobs= 1,
#  oob_score= True,
#  random_state= 0,
#  verbose= 0,
#  warm_start= False)

In [112]:
clf = RandomForestClassifier(bootstrap= True,
 class_weight= None,
 criterion= 'gini',
 max_depth= 20,
 max_features= 'auto',
 max_leaf_nodes= None,
 min_impurity_split= 1e-07,
 min_samples_leaf= 1,
 min_samples_split= 2,
 min_weight_fraction_leaf= 0.0,
 n_estimators= 150,
 n_jobs= 1,
 oob_score= True,
 random_state= 0,
 verbose= 0,
 warm_start= True)

In [113]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=True, random_state=0,
            verbose=0, warm_start=True)

In [114]:
print(clf.feature_importances_)

[ 0.00047146  0.00062218  0.00020499 ...,  0.00031227  0.00045035
  0.00064911]


In [115]:
y_pred = clf.predict(X_test)

In [116]:
clf.score(X,y)

0.95909376966645687

In [117]:
clf.oob_score_

0.9005663939584645

In [106]:
print_scores(y_test, y_pred)
# 150 trees
# max depth 10

Accuracy: 0.908957415565
Precision: 0.788194444444
Recall: 0.378333333333


In [96]:
print_scores(y_test, y_pred)
# 200 trees
# max depth 5

Accuracy: 0.903503251521
Precision: 0.833333333333
Recall: 0.291666666667


In [97]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted No Ideation', 'Predicted Ideation'],
    index=['True No Ideation', 'True Ideation']
)

,Predicted No Ideation,Predicted Ideation
True No Ideation,4132,35
True Ideation,425,175


In [89]:
print_scores(y_test, y_pred)
# 100 trees
# max depth 5

Accuracy: 0.903083700441
Precision: 0.828571428571
Recall: 0.29


In [111]:
print_scores(y_test, y_pred)
# 10 trees
# max depth 10
# gini
# warm start

Accuracy: 0.906230333543
Precision: 0.742857142857
Recall: 0.39


In [118]:
print_scores(y_test, y_pred)
# 150 trees
# max depth 20

Accuracy: 0.909376966646
Precision: 0.759259259259
Recall: 0.41


In [68]:
print_scores(y_test, y_pred)
# 150 trees
# entropy 

Accuracy: 0.907488986784
Precision: 0.777003484321
Recall: 0.371666666667


In [62]:
print_scores(y_test, y_pred)
# 150 trees

Accuracy: 0.909376966646
Precision: 0.789655172414
Recall: 0.381666666667


In [56]:
print_scores(y_test, y_pred)
# 100 trees

Accuracy: 0.908118313405
Precision: 0.771812080537
Recall: 0.383333333333


In [44]:
print_scores(y_test, y_pred)
# ORIGINAL SCORES 

Accuracy: 0.906230333543
Precision: 0.742857142857
Recall: 0.39


In [45]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted No Ideation', 'Predicted Ideation'],
    index=['True No Ideation', 'True Ideation']
)

,Predicted No Ideation,Predicted Ideation
True No Ideation,4086,81
True Ideation,366,234


In [82]:
f1_score(y_test, y_pred)

0.53347732181425489

## OOB Error

In [46]:
from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
RANDOM_STATE = 123

In [47]:
ensemble_clfs = [
    ("RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(warm_start=True, oob_score=True,
                               max_features="sqrt",
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(warm_start=True, max_features='log2',
                               oob_score=True,
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features=None",
        RandomForestClassifier(warm_start=True, max_features=None,
                               oob_score=True,
                               random_state=RANDOM_STATE))
]

In [48]:
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

In [49]:
min_estimators = 15
max_estimators = 175

In [50]:
for label, clf in ensemble_clfs:
    for i in range(min_estimators, max_estimators + 1):
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

/Users/Winnifred/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/Winnifred/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/Winnifred/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/Winnifred/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/Winnifred/anaconda2/envs/py3/lib/python3.6/site-p

KeyboardInterrupt: 

In [ ]:
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

## ROC Curve 

In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp


In [ ]:
# LOGISTIC REGRESSION SCORES 
# logreg_y = [0,1]
# logreg_scores = 

In [ ]:
# RANDOM FOREST SCORES 
# rf_y = [0,1]
# rf_scores = 

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)
for i in range(2):
    fpr, tpr, _ = roc_curve(y_test[:,i], y_score[:,i]) 
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)

In [ ]:
plt.figure()
plt.plot(fpr[2], tpr[2], color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', {'fontsize': 15})
plt.ylabel('True Positive Rate', {'fontsize': 15})
plt.title('ROC Comparison', {'fontsize': 20})
plt.legend(loc="lower right")
plt.show()